# InfoGenAI User Interface
This Jupyter Notebook implements a web user interface to InfoGenAI

In [1]:
!pip install gradio requests --quiet

In [ ]:
import gradio as gr
import requests
from datetime import datetime

def append_debug_message(debug_messages, message):
    """Appends a timestamped debug message to the debug log."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    debug_messages.append(f"{timestamp} D {message}")
    return debug_messages

def fetch_agents(backend_url, debug_messages):
    """Fetches the list of agents from the backend and logs the communication."""
    try:
        debug_messages = append_debug_message(debug_messages, f"Sending request to fetch agents: {backend_url}")
        response = requests.post(backend_url, data={"Body": "list_agents"})
        if response.status_code == 200:
            agents = response.text.split("\n")
            agent_descriptions = [line for line in agents if " - " in line]
            debug_messages = append_debug_message(debug_messages, f"Received agents: {agent_descriptions}")
            return agent_descriptions, debug_messages
        else:
            error_message = f"Error: Unable to fetch agents. Status code: {response.status_code}"
            debug_messages = append_debug_message(debug_messages, error_message)
            return [error_message], debug_messages
    except requests.RequestException as e:
        error_message = f"Request failed: {e}"
        debug_messages = append_debug_message(debug_messages, error_message)
        return [error_message], debug_messages

def analyze(prompt, backend_url, selected_agents, debug_messages):
    """Sends the article and selected agents to the backend for analysis and logs the communication."""
    try:
        selected_agents = [agent.split(" - ")[0] for agent in selected_agents if agent]
        debug_messages = append_debug_message(debug_messages, f"Sending analysis request with agents: {selected_agents}")
        response = requests.post(
            backend_url,
            data={"Body": prompt, "From": "gradio_user", "Agents": ",".join(selected_agents)},
        )
        if response.status_code == 200:
            debug_messages = append_debug_message(debug_messages, f"Received analysis response: {response.text}")
            return response.text, debug_messages
        else:
            error_message = f"Error: {response.status_code} - {response.text}"
            debug_messages = append_debug_message(debug_messages, error_message)
            return error_message, debug_messages
    except requests.RequestException as e:
        error_message = f"Request failed: {e}"
        debug_messages = append_debug_message(debug_messages, error_message)
        return error_message, debug_messages

with gr.Blocks() as interface:
    debug_messages = gr.State([])  # State to store debug messages

    with gr.Tab("Article"):
        # Backend Section
        with gr.Row():
            backend_url = gr.Textbox(label="Backend URL", value="http://localhost:5000/infofactagents", interactive=True)

        # Agents Section
        with gr.Row():
            enabled_agents = gr.CheckboxGroup(label="Enable Agents", choices=[], interactive=True)

        # Article Section
        with gr.Row():
            prompt = gr.Textbox(label="Enter news article", lines=5, interactive=True)

        # Analyze Button and Trustworthiness Output
        with gr.Row():
            send_button = gr.Button("Analyze")
        with gr.Row():
            output = gr.Textbox(label="Trustworthiness Analysis", lines=10, interactive=False)

        def update_agents(backend_url, debug_messages):
            agents, updated_debug = fetch_agents(backend_url, debug_messages)
            return gr.update(choices=agents), updated_debug
        
        # Automatically fetch agents when the interface starts
        interface.load(
            fn=update_agents,
            inputs=[backend_url, debug_messages],
            outputs=[enabled_agents, debug_messages],
        )
        
        send_button.click(
            fn=analyze,
            inputs=[prompt, backend_url, enabled_agents, debug_messages],
            outputs=[output, debug_messages],
        )
    
    with gr.Tab("Debug") as debug_tab:
        debug_output = gr.Textbox(label="Debug Info", lines=15, interactive=False)
        
        # Refresh debug info whenever the Debug tab is activated
        debug_tab.select(
            fn=lambda debug_messages: "\n".join(debug_messages),
            inputs=debug_messages,
            outputs=debug_output,
        )

interface.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
